# *THIS IS MY BEST REC MODEL*

# Setting up my full dataframe

In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import time
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

%matplotlib inline
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split

import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer

import gensim, logging, warnings
import gensim.corpora as corpora
from gensim import corpora, models
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df0 = pd.read_csv('data_files/scrape1.csv', index_col = 0)
df1 = pd.read_csv('data_files/scrape2.csv', index_col = 0)
df2 = pd.read_csv('data_files/scrape3.csv', index_col = 0)
df3 = pd.read_csv('data_files/scrape4.csv', index_col = 0)
df4 = pd.read_csv('data_files/scrape5.csv', index_col = 0)
df5 = pd.read_csv('data_files/scrape6.csv', index_col = 0)
df6 = pd.read_csv('data_files/scrape7.csv', index_col = 0)
df7 = pd.read_csv('data_files/scrape8.csv', index_col = 0)
df8 = pd.read_csv('data_files/scrape9.csv', index_col = 0)
df9 = pd.read_csv('data_files/scrape10.csv', index_col = 0)
df10 = pd.read_csv('data_files/scrape11.csv', index_col = 0)
df11 = pd.read_csv('data_files/scrape12.csv', index_col=0)
df12 = pd.read_csv('data_files/scrape13.csv', index_col=0)
df13 = pd.read_csv('data_files/scrape14.csv', index_col=0)
df14 = pd.read_csv('data_files/scrape15.csv', index_col=0)
df15 = pd.read_csv('data_files/scrape16.csv', index_col=0)
df16 = pd.read_csv('data_files/scrape17.csv', index_col=0)
df17 = pd.read_csv('data_files/scrape18.csv', index_col=0)

In [3]:
df = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17], axis=0)

In [4]:
df = df.reset_index(drop=True)

In [5]:
df.shape

(203126, 13)

In [8]:
df.sample(10)

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url
172401,4.4%,Year-round,3.53,NaN,Tirril Brewery,https://cdn.beeradvocate.com/im/placeholder-be...,England,Red Barn Ale,1,0,['3.53/5\xa0\xa0rDev 0%look: 4 | smell: 3.5 | ...,English Strong Ale,https://www.beeradvocate.com/beer/profile/3460...
60199,6.7%,Rotating,3.39,NaN,Fountainhead Brewing Company,https://cdn.beeradvocate.com/im/placeholder-be...,California,Fuzzy Notion,1,0,['3.39/5\xa0\xa0rDev 0%look: 3.5 | smell: 3.25...,American IPA,https://www.beeradvocate.com/beer/profile/5352...
44000,6.2%,Rotating,3.91,NaN,Fairfield Craft Ales,https://cdn.beeradvocate.com/im/placeholder-be...,Connecticut,Little Darlin IPA,4,0,['3.93/5\xa0\xa0rDev +0.5%look: 4.25 | smell: ...,American IPA,https://www.beeradvocate.com/beer/profile/4337...
8942,5.5%,Rotating,0.00,NaN,Benelux Brewpub & Café,https://cdn.beeradvocate.com/im/placeholder-be...,Quebec,L'ouvrière,0,0,[],American Brown Ale,https://www.beeradvocate.com/beer/profile/1380...
161056,10.5%,Fall,4.00,NaN,Brasserie De Metropole,https://cdn.beeradvocate.com/im/beers/114201.jpg,Russian Federation,Last Punch,1,0,"['4/5\xa0\xa0rDev 0%Gakusei, Feb 27, 2014']",Russian Imperial Stout,https://www.beeradvocate.com/beer/profile/2330...
27216,4.4%,Year-round,3.80,NaN,Siebensternbräu,https://cdn.beeradvocate.com/im/beers/30530.jpg,Austria,7 Stern Prager Dunkles,9,3,['4.03/5\xa0\xa0rDev +6.1%look: 4.5 | smell: 4...,Munich Dunkel Lager,https://www.beeradvocate.com/beer/profile/4098...
150655,6.2%,Rotating,3.78,NaN,Lariat Lodge Brewing Company,https://cdn.beeradvocate.com/im/placeholder-be...,Colorado,Squaw Pass Stout,1,0,['3.78/5\xa0\xa0rDev 0%look: 4.25 | smell: 3.7...,American Stout,https://www.beeradvocate.com/beer/profile/4321...
19346,4.7%,Year-round,3.13,NaN,BrewPub København,https://cdn.beeradvocate.com/im/placeholder-be...,Denmark,William Wallace,2,0,['3.25/5\xa0\xa0rDev +3.8%look: 3.25 | smell: ...,Scottish Ale,https://www.beeradvocate.com/beer/profile/1209...
36516,4%,Rotating,3.74,86.0,Fourpure Brewing Co.,https://cdn.beeradvocate.com/im/beers/340255.jpg,England,Easy Peeler,16,9,['3.68/5\xa0\xa0rDev -1.6%look: 3.5 | smell: 3...,American IPA,https://www.beeradvocate.com/beer/profile/3336...
37261,6.6%,Year-round,3.72,86.0,SeaQuake Brewing,https://cdn.beeradvocate.com/im/beers/274702.jpg,California,Citra IPA,13,6,['2.82/5\xa0\xa0rDev -24.2%look: 3.5 | smell: ...,American IPA,https://www.beeradvocate.com/beer/profile/4670...


In [10]:
len(df.loc[8942, 'review'])

2

In [15]:
def get_avg_taste(string):
    """takes in the review and pulls out the rating for the taste and returns an average"""
    list_list = [re.findall("\d{1}\.*\d*", item) for item in re.findall("taste: \d{1}\.*\d*", string)]
    sum_feel = 0
    tot = len(list_list)
    for list_item in list_list:
        for item in list_item:
            sum_feel += float(item)
    if not tot == 0:
        return sum_feel/tot
    else:
        return None


def get_avg_look(string):
    list_list = [re.findall("\d{1}\.*\d*", item) for item in re.findall("look: \d{1}\.*\d*", string)]
    sum_feel = 0
    tot = len(list_list)
    for list_item in list_list:
        for item in list_item:
            sum_feel += float(item)
    if not tot == 0:
        return sum_feel/tot
    else:
        return None

def get_avg_smell(string):
    list_list = [re.findall("\d{1}\.*\d*", item) for item in re.findall("smell: \d{1}\.*\d*", string)]
    sum_feel = 0
    tot = len(list_list)
    for list_item in list_list:
        for item in list_item:
            sum_feel += float(item)
    if not tot == 0:
        return sum_feel/tot
    else:
        return None


def get_avg_feel(string):
    list_list = [re.findall("\d{1}\.*\d*", item) for item in re.findall("feel: \d{1}\.*\d*", string)]
    sum_feel = 0
    tot = len(list_list)
    for list_item in list_list:
        for item in list_item:
            if item == '3..25':
                sum_feel += 3.25
            else:
                sum_feel += float(item)
    if not tot == 0:
        return sum_feel/tot
    else:
        return None

In [16]:
df['taste_avg'] = df['review'].apply(get_avg_taste)
df['look_avg'] = df['review'].apply(get_avg_look)
df['smell_avg'] = df['review'].apply(get_avg_smell)
df['feel_avg'] = df['review'].apply(get_avg_feel)

In [17]:
df.head()

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url,taste_avg,look_avg,smell_avg,feel_avg
0,5.6%,Winter,3.60,82.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Winter Lager,"6,210","2,048",['3.31/5\xa0\xa0rDev -8.1%look: 4 | smell: 3.5...,German Bock,https://www.beeradvocate.com/beer/profile/35/101/,3.56,3.76,3.48,3.63
1,4.4%,Year-round,3.19,73.0,Spoetzl Brewery,https://cdn.beeradvocate.com/im/beers/101.jpg,Texas,Shiner Bock,"4,331","1,163","[""3.19/5\xa0\xa0rDev 0%look: 3.75 | smell: 2.7...",German Bock,https://www.beeradvocate.com/beer/profile/143/...,3.47,3.76,3.31,3.43
2,5.8%,Rotating,3.66,83.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Chocolate Bock,"3,101","1,225",['3.78/5\xa0\xa0rDev +3.3%look: 3.75 | smell: ...,German Bock,https://www.beeradvocate.com/beer/profile/35/1...,3.65,3.64,3.59,3.61
3,5.2%,Year-round,2.89,68.0,Anheuser-Busch,https://cdn.beeradvocate.com/im/beers/101.jpg,Missouri,Michelob AmberBock,"1,906",707,"[""3.09/5\xa0\xa0rDev +6.9%look: 3.5 | smell: 2...",German Bock,https://www.beeradvocate.com/beer/profile/29/1...,3.24,3.62,3.27,3.21
4,5.8%,Rotating,3.50,80.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Cherry Chocolate Bock,873,187,['3/5\xa0\xa0rDev -14.3%look: 3 | smell: 3 | t...,German Bock,https://www.beeradvocate.com/beer/profile/35/6...,3.28,3.69,3.62,3.43


In [18]:
def clean_review(review):
    """takes in a review and does the following:
    1. removes the '\xa0'
    2. removes the '\n'
    3. performs the simple preprocess from gensim"""
    review = review.replace(u'\\xa0', '')
    review = review.replace('\\n', '')
    review = gensim.utils.simple_preprocess(str(review), deacc=True)
    review = [word for word in review if word not in stop_words]
    meta_lemmed = [lemmatizer.lemmatize(word) for word in review]
#     c = " ".join(str(x) for x in meta_lemmed)
#     cleaned.append(c)
    return meta_lemmed

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
stop_words.extend(['look', 'smell', 'taste', 'feel', 'overall', 'rdev', 'beer', 'beers','nice','adams', 'jan', 'feb', 'mar',
                  'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'ml', 'oz', 'date'])

In [6]:
df['clean_review'] = df['review'].apply(clean_review)

NameError: name 'df' is not defined

In [27]:
df.head()

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url,taste_avg,look_avg,smell_avg,feel_avg,clean_review
0,5.6%,Winter,3.60,82.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Winter Lager,"6,210","2,048",['3.31/5\xa0\xa0rDev -8.1%look: 4 | smell: 3.5...,German Bock,https://www.beeradvocate.com/beer/profile/35/101/,3.56,3.76,3.48,3.63,"[alright, one, depends, great, deep, copper, m..."
1,4.4%,Year-round,3.19,73.0,Spoetzl Brewery,https://cdn.beeradvocate.com/im/beers/101.jpg,Texas,Shiner Bock,"4,331","1,163","[""3.19/5\xa0\xa0rDev 0%look: 3.75 | smell: 2.7...",German Bock,https://www.beeradvocate.com/beer/profile/143/...,3.47,3.76,3.31,3.43,"[first, sampled, visited, father, texas, earli..."
2,5.8%,Rotating,3.66,83.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Chocolate Bock,"3,101","1,225",['3.78/5\xa0\xa0rDev +3.3%look: 3.75 | smell: ...,German Bock,https://www.beeradvocate.com/beer/profile/35/1...,3.65,3.64,3.59,3.61,"[black, color, little, light, penetration, fin..."
3,5.2%,Year-round,2.89,68.0,Anheuser-Busch,https://cdn.beeradvocate.com/im/beers/101.jpg,Missouri,Michelob AmberBock,"1,906",707,"[""3.09/5\xa0\xa0rDev +6.9%look: 3.5 | smell: 2...",German Bock,https://www.beeradvocate.com/beer/profile/29/1...,3.24,3.62,3.27,3.21,"[bottle, abv, purchased, cold, single, poured,..."
4,5.8%,Rotating,3.50,80.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Cherry Chocolate Bock,873,187,['3/5\xa0\xa0rDev -14.3%look: 3 | smell: 3 | t...,German Bock,https://www.beeradvocate.com/beer/profile/35/6...,3.28,3.69,3.62,3.43,"[initial, whiff, dr, pepper, cherry, dy, flavo..."


In [28]:
 # I might need to drop short reviews - do this on the cleaned reviews.
def drop_short_reviews(review):
    if len(review) < 10:
        return None
    else:
        return review

In [29]:
df['clean_review'] = df['clean_review'].apply(drop_short_reviews)

In [30]:
df['clean_review'].isna().sum()

93618

In [31]:
df.dropna(subset=['clean_review'], inplace=True)

In [32]:
df.shape

(109508, 18)

In [33]:
def to_string(list_):
    return " ".join(list_)

In [34]:
df['key_words'] = df['clean_review'].apply(to_string)

In [35]:
df.to_csv('full_no_null.csv')

# Here I apply my basic rec to the full dataset

In [8]:
df = pd.read_csv('full_no_null.csv', index_col = 0)

In [9]:
df.head()

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url,taste_avg,look_avg,smell_avg,feel_avg,clean_review,key_words
0,5.6%,Winter,3.60,82.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Winter Lager,"6,210","2,048",['3.31/5\xa0\xa0rDev -8.1%look: 4 | smell: 3.5...,German Bock,https://www.beeradvocate.com/beer/profile/35/101/,3.56,3.76,3.48,3.63,"['alright', 'one', 'depends', 'great', 'deep',...",alright one depends great deep copper molasses...
1,4.4%,Year-round,3.19,73.0,Spoetzl Brewery,https://cdn.beeradvocate.com/im/beers/101.jpg,Texas,Shiner Bock,"4,331","1,163","[""3.19/5\xa0\xa0rDev 0%look: 3.75 | smell: 2.7...",German Bock,https://www.beeradvocate.com/beer/profile/143/...,3.47,3.76,3.31,3.43,"['first', 'sampled', 'visited', 'father', 'tex...",first sampled visited father texas earlier yea...
2,5.8%,Rotating,3.66,83.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Chocolate Bock,"3,101","1,225",['3.78/5\xa0\xa0rDev +3.3%look: 3.75 | smell: ...,German Bock,https://www.beeradvocate.com/beer/profile/35/1...,3.65,3.64,3.59,3.61,"['black', 'color', 'little', 'light', 'penetra...",black color little light penetration fingernai...
3,5.2%,Year-round,2.89,68.0,Anheuser-Busch,https://cdn.beeradvocate.com/im/beers/101.jpg,Missouri,Michelob AmberBock,"1,906",707,"[""3.09/5\xa0\xa0rDev +6.9%look: 3.5 | smell: 2...",German Bock,https://www.beeradvocate.com/beer/profile/29/1...,3.24,3.62,3.27,3.21,"['bottle', 'abv', 'purchased', 'cold', 'single...",bottle abv purchased cold single poured fluted...
4,5.8%,Rotating,3.50,80.0,Boston Beer Company (Samuel Adams),https://cdn.beeradvocate.com/im/beers/101.jpg,Massachusetts,Samuel Adams Cherry Chocolate Bock,873,187,['3/5\xa0\xa0rDev -14.3%look: 3 | smell: 3 | t...,German Bock,https://www.beeradvocate.com/beer/profile/35/6...,3.28,3.69,3.62,3.43,"['initial', 'whiff', 'dr', 'pepper', 'cherry',...",initial whiff dr pepper cherry dy flavor give ...


In [10]:
df.describe()

,avg_score,ba_score,taste_avg,look_avg,smell_avg,feel_avg
count,109508.000000,41411.000000,108968.000000,108968.000000,108968.000000,108968.000000
mean,3.756687,85.116853,3.762992,3.827705,3.737850,3.748186
std,0.429753,5.716217,0.519760,0.439248,0.491403,0.491963
min,1.000000,42.000000,1.000000,1.000000,1.000000,1.000000
25%,3.560000,82.000000,3.500000,3.625000,3.500000,3.500000
50%,3.810000,86.000000,3.812500,3.890000,3.750000,3.778846
75%,4.020000,88.000000,4.060000,4.040000,4.000000,4.000000
max,5.000000,100.000000,5.000000,5.000000,5.000000,5.000000


In [24]:
sm_df_full = df.loc[df['avg_score'] > 3.9]

In [133]:
# change image to none if it's a placeholder
sm_df_full.loc[sm_df_full['img'] == "https://cdn.beeradvocate.com/im/placeholder-beer.jpg", 'img'] = None

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
sm_df.shape

(41869, 19)

In [13]:
sm_df['style'].nunique()

110

In [14]:
sm_df.drop(['abv', 'avail', 'avg_score', 'ba_score', 'brewery', 'img', 'location',
         'no_ratings', 'no_reviews', 'review', 'style', 'url',
       'taste_avg', 'look_avg', 'smell_avg', 'feel_avg', 'clean_review'], inplace=True, axis=1)

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [23]:
df.drop(['abv', 'avail', 'avg_score', 'ba_score', 'brewery', 'img', 'location',
         'no_ratings', 'no_reviews', 'review', 'style', 'url',
       'taste_avg', 'look_avg', 'smell_avg', 'feel_avg', 'clean_review'], inplace=True, axis=1)

In [15]:
sm_df.set_index(['name'], inplace=True, drop=True)

In [16]:
sm_df.sample(10)

,key_words
name,
Whispers In The Dark,tap despite warmer deep dark stout quite enjoy...
Shaka Mosaic,light lemony inviting grainy note first soon s...
East Rock Lager,crystal clear medium gold color fairly aggress...
Oktoberfest,clear little head brownish amber color lightly...
Belgian Tripel,tap poured clear golden amber color small fluf...
Red Legs Scotch Ale,deep mahogany red hue thin yet creamy head she...
Neverender,picked nearly year ago seeing austin street ba...
Bavarian Almond Bros,pours orangish color semi murky finger white h...
Montessori,decervecitas teku glass playing avernum fruity...


In [17]:
stop_words.extend(['alright', 'one', 'pour', 'pours', 'gluten', 'made', 'glass', 'one', 'bit', 'maybe', 'pretty',
                  'served', 'key', 'keg', 'good', 'wonder', 'mix', 'left', 'inch', 'american', 'style', 'similar'])

In [121]:
# instantiating and generating the count matrix
count = CountVectorizer(stop_words=stop_words, max_df=0.7, min_df=4, analyzer='word')
count_matrix = count.fit_transform(sm_df['key_words'])
count_matrix.shape
# try min_df or max_df

(41869, 30278)

In [123]:
print(count_matrix.get_feature_names())

AttributeError: get_feature_names not found

In [40]:
import pickle
pickle.dump(count_matrix, open('count_matrix_tiny.p', 'wb'))

In [55]:
# or we could do tfidf:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1),max_df=.7, min_df=4, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(sm_df['key_words'])
tfidf_matrix.shape

(41869, 30278)

In [ ]:
%time tfidf_cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [19]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [48]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(sm_df.index)

#  defining the function that takes in beer name 
# as input and returns the top 10 recommended beers
def recommendations(cosine_sim = cosine_sim):
    title = input('What beer do you like?')
    # initializing the empty list of recommended beers
    recommended_beers = []
    
    # gettin the index of the beer that matches the name
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching beers
    for i in top_10_indexes:
        recommended_beers.append(list(sm_df.index)[i])
        
    return recommended_beers

In [23]:
recommendations()

What beer do you like?Imperial Black IPA


['Badass Goat Black IPA',
 'Midnight Showers',
 'Blackcyl',
 'Black Rajah',
 'Eddyline Black Eddy Black IPA',
 'Chocolate Milkshake IPA',
 'Epitome',
 'In The Dark We Live',
 'Don Kiote',
 'Close Encounter']

In [53]:
recommendations()

What beer do you like?Game Over - Plum Cranberry


['Game Over - Cherry Berry',
 'Cranberry Sour',
 'Auroral',
 'Cape Codder Weisse',
 'Autre Grand Rouge',
 'Game Over - Double Plum w/ Ceylon Cinnamon',
 'Press Tart',
 'Frucht: Cranberry And Orange',
 'Dbl Underscore On The Beach',
 'Boysen Weisse']

In [138]:
# creating a Series for the beers so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(sm_df.index)

#  defining the function that takes in beer name 
# as input and returns the top 3 recommended beers
def full_recommendations(cosine_sim = cosine_sim):
    title = input('What beer do you like?')
    filter_by = input('Do you want to filter by location?')
    if filter_by == 'Yes':
        location = input('Where are you?')
    # initializing the empty list of recommended beers
    recommended_beers = []
    
    # gettin the index of the beer that matches the name
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_20_indexes = list(score_series.iloc[1:21].index)
    
    # populating the list with the titles of the best 10 matching beers
    
    
    if filter_by == 'No':
        
        for i in top_20_indexes:
            rec_beer_dict = {}
            full_ind = sm_df_full.index[sm_df_full['name'] == list(sm_df.index)[i]].tolist()[0]
            rec_beer_dict['Beer'] = list(sm_df.index)[i]
            rec_beer_dict['Brewery'] = sm_df_full.loc[full_ind]['brewery']
            rec_beer_dict['Location'] = sm_df_full.loc[full_ind]['location']
            rec_beer_dict['Rating'] = sm_df_full.loc[full_ind]['avg_score']
            rec_beer_dict['url'] = sm_df_full.loc[full_ind]['url']
            rec_beer_dict['Image'] = sm_df_full.loc[full_ind]['img']
    #         if sm_df_full.loc[full_ind]['location'] == location:
            recommended_beers.append(rec_beer_dict)
        print('--------------------')
        print('Then you should try:')    
        return recommended_beers[:3]
    
    elif filter_by == 'Yes':
        for i in top_20_indexes:
            rec_beer_dict = {}
            full_ind = sm_df_full.index[sm_df_full['name'] == list(sm_df.index)[i]].tolist()[0]
            rec_beer_dict['Beer'] = list(sm_df.index)[i]
            rec_beer_dict['Brewery'] = sm_df_full.loc[full_ind]['brewery']
            rec_beer_dict['Location'] = sm_df_full.loc[full_ind]['location']
            rec_beer_dict['Rating'] = sm_df_full.loc[full_ind]['avg_score']
            rec_beer_dict['url'] = sm_df_full.loc[full_ind]['url']
            rec_beer_dict['Image'] = sm_df_full.loc[full_ind]['img']
            if sm_df_full.loc[full_ind]['location'] == location:
                recommended_beers.append(rec_beer_dict)
    
        print('--------------------')
        print('Then you should try:')    
        return recommended_beers

In [90]:
full_recommendations()

What beer do you like?La Trappe Bockbier
--------------------
Then you should try:


[{'Beer': 'Vicaris Generaal',
  'Location': 'Belgium',
  'Rating': 3.96,
  'url': 'https://www.beeradvocate.com/beer/profile/21734/26214/'},
 {'Beer': 'Loterbol Bruin',
  'Location': 'Belgium',
  'Rating': 4.04,
  'url': 'https://www.beeradvocate.com/beer/profile/5221/11743/'},
 {'Beer': 'Gouden Carolus Classic',
  'Location': 'Belgium',
  'Rating': 4.14,
  'url': 'https://www.beeradvocate.com/beer/profile/1534/4106/'}]

In [148]:
full_recommendations()

What beer do you like?Coconut Pastry Stout
Do you want to filter by location?Yes
Where are you?New York
--------------------
Then you should try:


[{'Beer': 'Coffee Coconut Icing On The Cake',
  'Brewery': 'Grimm Artisanal Ales',
  'Location': 'New York',
  'Rating': 4.15,
  'url': 'https://www.beeradvocate.com/beer/profile/32893/399350/',
  'Image': 'https://cdn.beeradvocate.com/im/beers/399350.jpg'},
 {'Beer': "Scusa, Ma Parlo Solo Un Po' Di Macaroon Coconut",
  'Brewery': 'Evil Twin Brewing NYC',
  'Location': 'New York',
  'Rating': 4.43,
  'url': 'https://www.beeradvocate.com/beer/profile/55267/420797/',
  'Image': 'https://cdn.beeradvocate.com/im/beers/420797.jpg'}]

In [134]:
full_recommendations()

What beer do you like?Morning Wood
Do you want to filter by location?No
--------------------
Then you should try:


[{'Beer': 'Wrapped in Simcoe',
  'Brewery': 'Twisted Barrel Ale',
  'Location': 'England',
  'Rating': 4.3,
  'url': 'https://www.beeradvocate.com/beer/profile/45563/333675/',
  'Image': 'https://cdn.beeradvocate.com/im/beers/333675.jpg'},
 {'Beer': 'Kwas Epsilon Wild & Barrel Aged #1',
  'Brewery': 'Browar Pinta',
  'Location': 'Poland',
  'Rating': 4.4,
  'url': 'https://www.beeradvocate.com/beer/profile/29347/374248/',
  'Image': None},
 {'Beer': 'Gehenna Merlot Barrel Aged',
  'Brewery': 'Tempel Brygghus',
  'Location': 'Sweden',
  'Rating': 4.35,
  'url': 'https://www.beeradvocate.com/beer/profile/42789/226510/',
  'Image': None}]

In [125]:
sm_df.sample(20)

,key_words
name,
Hop Therapy ISA,suffers fate many isas reduction malt required...
Wild Sinister Kid (Black Currants),big thanks boi dingo hooking one dark purplish...
Back on the Train,jeff thanks opportunity hazy muddy dark yellow...
Three Fates,outstanding around find much complain thing la...
Vermilion Barleywine,poured teku glass somewhat hazy copper brown c...
XOXO Coffee Stout,interesting coffee stout distinct coffee flavo...
Barrel Thieves and Hired Guns,imperial stout brewed molasses vanilla chocola...
Isoelectric Shift,benchtop brewing co isoelectric shift tap brew...
Flight Of The UFOnaut,tap ashley westland short v greenbush throwdow...


In [ ]:
Pennsylvania
Coconut Pastry Stout

In [37]:
list(sm_df.index)[1]

'La Trappe Bockbier'

In [67]:
sm_df_full.loc[53573]['location']

'Washington'

In [71]:
sm_df_full.index[sm_df_full['name'] == 'La Trappe Bockbier'].tolist()[0]

12

In [68]:
sm_df_full.index 

Int64Index([     7,     12,     17,     41,     46,     63,     70,     72,
                88,     89,
            ...
            202483, 202484, 202493, 202494, 202504, 202505, 202524, 202528,
            202531, 202538],
           dtype='int64', length=41869)

In [66]:
str(location)

'53573    Washington\nName: location, dtype: object'

In [ ]:
sm_df_full

In [80]:
sm_df_full.sample(10)

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url,taste_avg,look_avg,smell_avg,feel_avg,clean_review,key_words
150501,5.7%,Rotating,4.12,NaN,Noble Shepherd Craft Brewery,https://cdn.beeradvocate.com/im/placeholder-be...,New York,Coconut Pastry Stout,1,1,['4.12/5\xa0\xa0rDev 0%look: 4.25 | smell: 4 |...,American Stout,https://www.beeradvocate.com/beer/profile/4323...,4.250000,4.250000,4.000000,4.000000,"['tap', 'noble', 'shepherd', 'craft', 'brewery...",tap noble shepherd craft brewery bloomfield ny...
159597,10%,Year-round,3.96,90.0,Ridgeway Brewing,https://cdn.beeradvocate.com/im/beers/147203.jpg,England,Imperial Russian Stout,29,15,['4.09/5\xa0\xa0rDev +3.3%look: 4 | smell: 4.2...,Russian Imperial Stout,https://www.beeradvocate.com/beer/profile/7944...,4.017857,4.017857,3.964286,3.875000,"['vintage', 'year', 'age', 'bottlepours', 'bla...",vintage year age bottlepours black tan colored...
160091,9%,Rotating,3.92,NaN,Prohibition Pig,https://cdn.beeradvocate.com/im/placeholder-be...,Vermont,Drago,7,3,"[""4.18/5\xa0\xa0rDev +6.6%look: 4.5 | smell: 4...",Russian Imperial Stout,https://www.beeradvocate.com/beer/profile/2866...,4.125000,4.187500,3.687500,3.937500,"['second', 'bier', 'dinner', 'visit', 'alewife...",second bier dinner visit alewife pour poured j...
180030,5.5%,One-off,3.92,NaN,More Brewing Co.,https://cdn.beeradvocate.com/im/placeholder-be...,Illinois,Zesty Enterprise: Raspberry & Meyer Lemon,4,2,"[""3.97/5\xa0\xa0rDev +1.3%look: 4.5 | smell: 3...",Berliner Weisse,https://www.beeradvocate.com/beer/profile/4974...,4.000000,4.125000,3.875000,3.875000,"['vibrant', 'ruby', 'purple', 'red', 'creamy',...",vibrant ruby purple red creamy pepto pink ring...
37708,5.7%,Rotating,3.93,87.0,Jackie O's Pub & Brewery,https://cdn.beeradvocate.com/im/placeholder-be...,Ohio,Alphanox,12,0,['3.87/5\xa0\xa0rDev -1.5%look: 4.25 | smell: ...,American IPA,https://www.beeradvocate.com/beer/profile/1432...,4.035714,4.000000,4.142857,4.107143,"['dangerdawg', 'drbenderdo', 'matt', 'zimm', '...",dangerdawg drbenderdo matt zimm wvubrew alexfi...
195203,6.2%,Rotating,4.21,NaN,Noble Creature Cask House,https://cdn.beeradvocate.com/im/placeholder-be...,Ohio,Chelsette,1,1,['4.21/5\xa0\xa0rDev 0%look: 4 | smell: 4.25 |...,American Wild Ale,https://www.beeradvocate.com/beer/profile/5167...,4.250000,4.000000,4.250000,4.000000,"['burnt', 'chestnut', 'brown', 'fuzzy', 'tan',...",burnt chestnut brown fuzzy tan head quickly dy...
35363,6.8%,Rotating,4.01,88.0,Offshoot Beer Co.,https://cdn.beeradvocate.com/im/beers/315745.jpg,California,Hold My Beer,24,5,['4.34/5\xa0\xa0rDev +8.2%look: 4 | smell: 4.2...,American IPA,https://www.beeradvocate.com/beer/profile/4878...,4.000000,3.700000,4.100000,3.900000,"['tap', 'lure', 'bar', 'grille', 'favorite', '...",tap lure bar grille favorite ipa dipas ton jui...
168000,9.2%,Year-round,4.00,87.0,Reformation Brewery,https://cdn.beeradvocate.com/im/placeholder-be...,Georgia,Jude,52,15,['3.47/5\xa0\xa0rDev -13.3%look: 4 | smell: 3....,Belgian Tripel,https://www.beeradvocate.com/beer/profile/3352...,4.066667,4.033333,3.916667,3.916667,"['pours', 'dull', 'hazy', 'orange', 'nose', 'm...",pours dull hazy orange nose mild fruity ester ...
155134,13.5%,Rotating,4.20,90.0,Pollyanna Brewing Company,https://cdn.beeradvocate.com/im/beers/331085.jpg,Illinois,Imperial Fun Size - Bourbon Barrel-Aged,29,8,['4.35/5\xa0\xa0rDev +3.6%look: 4.25 | smell: ...,English Sweet / Milk Stout,https://www.beeradvocate.com/beer/profile/3450...,4.156250,4.187500,4.218750,4.031250,"['black', 'tan', 'headsmell', 'smell', 'super'...",black tan headsmell smell super sweettaste wow...
53508,7.1%,One-off,3.96,NaN,Yellow Dog Brewing,https://cdn.beeradvocate.com/im/beers/446480.jpg,British Columbia,Class of 2014 IPA,1,1,"[""3.96/5\xa0\xa0rDev 0%look: 4.25 | smell: 3.7...",American IPA,https://www.beeradvocate.com/beer/profile/3591...,4.000000,4.250000,3.750000,4.000000,"['poured', 'ibu', 'brewed', 'bc', 'craft', 'mo

In [76]:
sm_df.head()

,key_words
name,
St. Nikolaus Bock Bier,dark black slight tan head roasted barley good...
La Trappe Bockbier,deep chestnut ruby slight haze probably bottle...
Schokolade Bock,review german pilsner figure delete erroneous ...
Aass Bock,light brown pour earthy edge thin head white a...
Poop Your Pants Chocolate Bock,including tax six pack meijer birch run mi und...


In [111]:
sm_df_full.loc[sm_df_full['name'] == 'Coconut Pastry Stout']

,abv,avail,avg_score,ba_score,brewery,img,location,name,no_ratings,no_reviews,review,style,url,taste_avg,look_avg,smell_avg,feel_avg,clean_review,key_words
150501,5.7%,Rotating,4.12,NaN,Noble Shepherd Craft Brewery,https://cdn.beeradvocate.com/im/placeholder-be...,New York,Coconut Pastry Stout,1,1,['4.12/5\xa0\xa0rDev 0%look: 4.25 | smell: 4 |...,American Stout,https://www.beeradvocate.com/beer/profile/4323...,4.25,4.25,4.0,4.0,"['tap', 'noble', 'shepherd', 'craft', 'brewery...",tap noble shepherd craft brewery bloomfield ny...


In [135]:
sm_df_full.to_csv('top_beers.csv')

In [146]:
def fun_recommendations(cosine_sim = cosine_sim):
    title = input('What beer do you like?')
    if title == "Budweiser":
        print("You would enjoy water")
    else:
        filter_by = input('Do you want to filter by location?')
    
        if filter_by == 'Yes':
            location = input('Where are you?')
    # initializing the empty list of recommended beers
    recommended_beers = []
    
    # gettin the index of the beer that matches the name
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_20_indexes = list(score_series.iloc[1:21].index)
    
    # populating the list with the titles of the best 10 matching beers
    
    
    if filter_by == 'No':
        for i in top_20_indexes:
            rec_beer_dict = {}
            full_ind = sm_df_full.index[sm_df_full['name'] == list(sm_df.index)[i]].tolist()[0]
            rec_beer_dict['Beer'] = list(sm_df.index)[i]
            rec_beer_dict['Brewery'] = sm_df_full.loc[full_ind]['brewery']
            rec_beer_dict['Location'] = sm_df_full.loc[full_ind]['location']
            rec_beer_dict['Rating'] = sm_df_full.loc[full_ind]['avg_score']
            rec_beer_dict['url'] = sm_df_full.loc[full_ind]['url']
            rec_beer_dict['Image'] = sm_df_full.loc[full_ind]['img']
        #         if sm_df_full.loc[full_ind]['location'] == location:
            recommended_beers.append(rec_beer_dict)
        print('--------------------')
        print('Then you should try:')    
        return recommended_beers[:3]
    
    elif filter_by == 'Yes':
        for i in top_20_indexes:
            rec_beer_dict = {}
            full_ind = sm_df_full.index[sm_df_full['name'] == list(sm_df.index)[i]].tolist()[0]
            rec_beer_dict['Beer'] = list(sm_df.index)[i]
            rec_beer_dict['Brewery'] = sm_df_full.loc[full_ind]['brewery']
            rec_beer_dict['Location'] = sm_df_full.loc[full_ind]['location']
            rec_beer_dict['Rating'] = sm_df_full.loc[full_ind]['avg_score']
            rec_beer_dict['url'] = sm_df_full.loc[full_ind]['url']
            rec_beer_dict['Image'] = sm_df_full.loc[full_ind]['img']
            if sm_df_full.loc[full_ind]['location'] == location:
                recommended_beers.append(rec_beer_dict)
    
        print('--------------------')
        print('Then you should try:')    
        return recommended_beers

In [147]:
fun_recommendations()

What beer do you like?Budweiser
You would enjoy water


IndexError: index 0 is out of bounds for axis 0 with size 0